In [1]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from datetime import datetime

In [2]:
# Threshold to take number of genes (biomarkers) per subtype
THRESHOLD = 10
COHORT = 'TCGA_BRCA'
assert COHORT in ['TCGA_BRCA','TCGA_LUNG','TCGA_CRC','METABRIC']
RANDOM_STATE = 42

In [3]:
LIST_OMICS = ['GE', 'CNA']
# Single omic or Multi-omics| to run experiments
LIST_EXP_OMICS = ['GE_CNA','GE', 'CNA']

ROOT_DATA_FOLDER = f'../input/deepssc-omics-pretrained-model-and-biomarkers/DeepSSC_data/{COHORT}/'
DATA_FOLDER = {'train': ROOT_DATA_FOLDER + f'data/train/',
              'test': ROOT_DATA_FOLDER + f'data/test/'}
BIOMARKER_FILE = ROOT_DATA_FOLDER + f'biomarkers/matrix_biomarkers.csv'
if COHORT == 'METABRIC':
    HGNC_SYMBOL_CHECK_FILE = ROOT_DATA_FOLDER + f'biomarkers/hgnc_symbol_check.csv'
    # for matching gene name between two cohort
LIST_TYPE_DATA = ['train', 'test']

In [4]:
# Read data
dict_df_label = {}
dict_df_data = {}
# Read data as df and create numpy array data for labeled data
for type_data in LIST_TYPE_DATA:
    dict_df_label[type_data] = pd.read_csv(DATA_FOLDER[type_data] + f'df_label_{type_data}.csv', index_col='sampleID')
    
    dict_df_omics = {}
    dict_narray_omics = {}
    for omic in LIST_OMICS:
        dict_df_omics[omic] = pd.read_csv(DATA_FOLDER[type_data] + f'df_{type_data}_{omic}_labeled.csv', index_col='sampleID')
    dict_df_data[type_data] = dict_df_omics

LABEL_MAPPING_NAME = dict_df_label['train']['disease_subtypes'].astype('category').cat.categories # sorted by alphabetical order
# Convert categorical label to numerical label
for type_data in LIST_TYPE_DATA:
    dict_df_label[type_data].loc[:,'disease_subtypes'] = dict_df_label[type_data]['disease_subtypes'].astype('category').cat.codes

#---------------------------------------------------------------------------------------
# Keep only biomarker genes found from TCGA data
print('-'*100)
print('KEEP ONLY BIOMARKER GENES FOUND FROM TCGA DATA')
score_genes = pd.read_csv(BIOMARKER_FILE)

score_genes = score_genes.iloc[:THRESHOLD]
top_genes = list(set(score_genes.to_numpy(copy=True).reshape(-1)))
top_genes = [gene.upper() for gene in top_genes]
print(f'Top {THRESHOLD} from TCGA have {len(top_genes)} unique genes')

if COHORT == 'METABRIC':
    # Check missing genes in METABRIC
    origin_unique_gene = {}
    for omic in LIST_OMICS:
        origin_unique_gene[omic] = list(dict_df_data['train'][omic].columns.str.upper().str.split(r'\|').str[0].unique())

    print('\nBefore matching alias gene names ')
    missing_genes = {}
    for omic in LIST_OMICS: 
        print(f'\tmissing {omic}:')
        missing_genes[omic] = list(set(top_genes) - set(origin_unique_gene[omic]))
        print('\t\t Number of missing',len(missing_genes[omic]))

    print('\tmissing both GE and CNA:')
    missing_genes_both_GE_CNA = list(set(missing_genes['GE']).intersection(set(missing_genes['CNA'])))
    print('\t\t Number of missing ',len(missing_genes_both_GE_CNA))
    print('\t\t',missing_genes_both_GE_CNA)


    print('\nAfter matching alias gene names ')
    alias_gene_names = pd.read_csv(HGNC_SYMBOL_CHECK_FILE,index_col='Input')
    alias_gene_names.index = alias_gene_names.index.str.upper()
    alias_gene_names['Approved symbol'] = alias_gene_names['Approved symbol'].str.upper()

    alias_gene_names = alias_gene_names.loc[alias_gene_names.index.isin(missing_genes_both_GE_CNA)]
    alias_gene_names = alias_gene_names.loc[alias_gene_names['Approved symbol'].notna(),['Approved symbol']]

    # Match alias gene name TCGA on METABRIC
    top_genes_fixed = []
    list_input_alias_gene_names =  alias_gene_names.index.tolist()
    for gene_name in top_genes:
        if gene_name in list_input_alias_gene_names:
            # change gene_name to its' alias gene names that can be used in METABRIC
            alias_names = alias_gene_names.loc[gene_name]['Approved symbol']
            if isinstance(alias_names, str):
                top_genes_fixed.append(alias_names)
            else:
                # an genes can have more than 1 suitable alias gene names
                top_genes_fixed.extend(alias_names.tolist())
        else:
            # gene name is existed in METABRIC or cannot find any suitable alias gene names
            # can be used in METABRIC
            top_genes_fixed.append(gene_name)
    print(f'Total top_genes_fixed {len(top_genes_fixed)}')

    missing_genes = {}
    for omic in LIST_OMICS: 
        print(f'\tmissing {omic}:')
        missing_genes[omic] = list(set(top_genes_fixed) - set(origin_unique_gene[omic]))
        print('\t\t Number of missing',len(missing_genes[omic]))

    print('\tmissing both GE and CNA:')
    missing_genes_both_GE_CNA = list(set(missing_genes['GE']).intersection(set(missing_genes['CNA'])))
    print('\t\t Number of missing ',len(missing_genes_both_GE_CNA))
    print('\t\t',missing_genes_both_GE_CNA)

    top_genes = top_genes_fixed

GENE = {}
print("\nNum features/genes:")
for omic in LIST_OMICS:
    GENE[omic] = dict_df_data['train'][omic].columns[
        dict_df_data['train'][omic].columns.str.upper().str.split(r'\|').str[0].isin(top_genes)
    ].to_numpy(copy=True).tolist()

    print(f'\twith {omic} TOP {THRESHOLD}:', len(GENE[omic]))
    
    for type_data in LIST_TYPE_DATA:
        dict_df_data[type_data][omic] = dict_df_data[type_data][omic][GENE[omic]].copy(deep=True)

if COHORT == 'METABRIC':
    # Fill missing values with mode value
    for type_data in LIST_TYPE_DATA:
        print(type_data)
        for omic in LIST_OMICS:
            print(omic)
            print('\tBefore imputing:')
            iloc_null = np.argwhere(np.isnan(dict_df_data[type_data][omic].to_numpy(copy=True)))
            iloc_gene_col_null = np.unique(iloc_null[:,1])
            print(f'\t\tNumber of null: {len(iloc_null)}')
            print(f'\t\tNumber of genes have null: {len(iloc_gene_col_null)}')
            
            if len(iloc_null) > 0:
                print('\tImputing:')
                for index in iloc_gene_col_null:
                    dict_df_data[type_data][omic].iloc[:,index].fillna(
                        dict_df_data[type_data][omic].iloc[:,index].mode()[0], inplace=True
                    )
                print('\tAfter imputing:')
                iloc_null = np.argwhere(np.isnan(dict_df_data[type_data][omic].to_numpy(copy=True)))
                print(f'\t\tNumber of null: {len(iloc_null)}')

----------------------------------------------------------------------------------------------------
KEEP ONLY BIOMARKER GENES FOUND FROM TCGA DATA
Top 10 from TCGA have 46 unique genes

Num features/genes:
	with GE TOP 10: 46
	with CNA TOP 10: 15


In [5]:
dict_X = {}
dict_y = {}
for type_data in LIST_TYPE_DATA:
    dict_X[type_data] = {}
    dict_y[type_data] = {}

for type_omic in LIST_EXP_OMICS:
    if '_' in type_omic:
        print(f'Creating data for multi-omics experiment: {type_omic}')
        list_omics = type_omic.split('_')
        for type_data in LIST_TYPE_DATA:
            tuple_data_omics = tuple([dict_df_data[type_data][single_omic] for single_omic in list_omics])
            dict_X[type_data][type_omic] = np.concatenate(tuple_data_omics, axis=1)
    else:
        print(f'Creating data for single omic experiment: {type_omic}')
        for type_data in LIST_TYPE_DATA:
            dict_X[type_data][type_omic] = dict_df_data[type_data][type_omic].to_numpy(copy=True)
    
    for type_data in LIST_TYPE_DATA:
        dict_y[type_data][type_omic] = dict_df_label[type_data]['disease_subtypes'].to_numpy(copy=True)

Creating data for multi-omics experiment: GE_CNA
Creating data for single omic experiment: GE
Creating data for single omic experiment: CNA


In [6]:
from IPython.display import Markdown, display
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)
def printmd(string, color=None):
    colorstr = "<span style='color:{}'>{}</span>".format(color, string)
    display(Markdown(colorstr))

In [7]:
def tuning_and_eval(gridcvs, X_train, y_train, X_test, y_test,
                    scoring, refit, is_binary_problem,
                    result_on_dataset, rank_hparams_info):
    assert 'test' in result_on_dataset
    assert isinstance(rank_hparams_info, bool)

    start=datetime.now()
    X = {}
    y = {}
    X['train'] = X_train
    X['test'] = X_test
    y['train'] = y_train
    y['test'] = y_test
    
    for name, gs_est in sorted(gridcvs.items()):
        printmd(f'{name} classifier:\n', color="blue")
        gs_est.fit(X['train'],y['train'])
        print(f'%s | best {refit} score  %.2f%% +/- %.2f' % 
          (name, gs_est.best_score_ * 100, gs_est.cv_results_[f'std_test_{refit}'][gs_est.best_index_] * 100))

        print('best parmams:', gs_est.best_params_)
        if rank_hparams_info:
            print('\n')
            select_result_cols = []
            for metric in scoring:
                select_result_cols.extend(['rank_test_'+metric,'mean_test_'+ metric, 'std_test_'+metric])
            select_result_cols.extend(['params'])
            
            dataframe_results = pd.DataFrame(gs_est.cv_results_).loc[:,select_result_cols].sort_values(by=f'mean_test_{refit}',ascending=False)
            display(dataframe_results[:10])

        for type_data in result_on_dataset:
            print('\n')
            print(f'Result on {type_data} dataset with best hyperparameters:')
            y_predict = gs_est.predict(X[type_data])
            
            acc = accuracy_score(y_true=y[type_data], y_pred=y_predict)
            print(f'{type_data}_acc: {acc * 100:.2f}')
            if is_binary_problem:
                f1 = f1_score(y_true=y[type_data], y_pred=y_predict,average='binary')
                y_score = gs_est.predict_proba(X[type_data])[:, 1]
                roc_auc = roc_auc_score(y_true=y[type_data], y_score=y_score)
                print(f'{type_data}_f1: {f1 * 100:.2f}')
                print(f'{type_data}_roc_auc: {roc_auc * 100:.2f}')
            else:
                f1_macro = f1_score(y_true=y[type_data], y_pred=y_predict,average='macro')
                f1_weighted = f1_score(y_true=y[type_data], y_pred=y_predict,average='weighted')
                print(f'{type_data}_f1_macro: {f1_macro * 100:.2f}')
                print(f'{type_data}_f1_weighted: {f1_weighted * 100:.2f}')
                
            display(pd.DataFrame(classification_report(y_true=y[type_data], 
                                                       y_pred=y_predict,
                                                       digits=4, output_dict=True)))
            display(pd.crosstab(y[type_data],y_predict,margins=True))
            print('-'*100)
    print(f'Total Time: {datetime.now()-start}')

In [8]:
def validate_biomarker(dict_X_train, dict_y_train, dict_X_test, dict_y_test,
                       omics=['GE_CNA', 'GE','CNA'], random_state = RANDOM_STATE,
                       result_on_dataset = ['train','test'], rank_hparams_info = True,
                       is_binary_problem=False):
    assert 'test' in result_on_dataset
    assert isinstance(rank_hparams_info, bool)
    
    scoring = None
    refit= None
    if is_binary_problem:
        scoring = ['f1','accuracy','roc_auc']
        refit = 'f1'
    else: 
        scoring = ['f1_macro','f1_weighted', 'accuracy']
        refit = 'f1_macro'
        
    # Initializing classifiers
    clf1 = LogisticRegression(random_state=random_state)

    # Binary case, probability = True to cal ROC_AUC, slowdown k-fold....
    clf2 = SVC(random_state=random_state, probability=is_binary_problem)

    clf3 = RandomForestClassifier(random_state=random_state)

    # Setting up the parameter grids
    param_grid1 = [{
                    'penalty': ['l2'],
                    'multi_class':["multinomial"],
                    'solver':["newton-cg"],
                    'class_weight': ["balanced"],
                    'C': np.power(10., np.arange(-4, 3)),
                    }]

    param_grid2 = [{
                    'kernel': ['rbf'],
                    'class_weight': ["balanced"],
                    'C': np.power(10., np.arange(-4, 3)),
                    'gamma': list(np.power(10., np.arange(-4, 0))) + ['scale']
                    }]

    param_grid3 = [{'n_estimators': [50, 100, 150],
                    'max_features': ["sqrt", "log2"],
                    'max_depth' : list(range(1, 10)) + [None],
                    'criterion' :["gini", "entropy"],
                    'class_weight': ["balanced", "balanced_subsample"]}]

    # Setting up multiple GridSearchCV objects, 1 for each algorithm
    gridcvs = {}
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=random_state)

    train_options = zip((param_grid1
                         ,param_grid2\
                         ,param_grid3
                        ),
                        (clf1,clf2\
                         ,clf3
                        ),
                        ('1_Softmax', '2_SVM'\
                         ,'3_RandomForest'
                        ))

    for pgrid, est, name in train_options:
        gcv = GridSearchCV(estimator=est,
                           param_grid=pgrid,
                           scoring=scoring,
                           n_jobs=-1,
                           cv=cv,
                           verbose=1,
                           refit=refit)
        gridcvs[name] = gcv
    
    for omic in omics:
        print('-'*100)
        printmd(f'Validate on {omic} data:\n', color="red")

        X_train = dict_X_train[omic]
        y_train = dict_y_train[omic]
        print('Train dist: ', np.unique(y_train, return_counts=True ))

        X_test = dict_X_test[omic]
        y_test = dict_y_test[omic]
        print('Test dist', np.unique(y_test, return_counts=True ),'\n')
        
        # run tuning and eval
        tuning_and_eval(gridcvs, X_train, y_train, X_test, y_test,\
                        scoring, refit,is_binary_problem,
                        result_on_dataset, rank_hparams_info)

In [9]:
validate_biomarker(dict_X['train'], dict_y['train'], dict_X['test'], dict_y['test'],
                   omics=LIST_EXP_OMICS, random_state=RANDOM_STATE,
                   result_on_dataset= ['test'], rank_hparams_info =False,
                   is_binary_problem = (len(LABEL_MAPPING_NAME)==2))

----------------------------------------------------------------------------------------------------


<span style='color:red'>Validate on GE_CNA data:
</span>

Train dist:  (array([0, 1, 2, 3, 4], dtype=int8), array([106,  55, 336, 142,  17]))
Test dist (array([0, 1, 2, 3, 4], dtype=int8), array([28, 12, 73, 45,  5])) 



<span style='color:blue'>1_Softmax classifier:
</span>

Fitting 50 folds for each of 7 candidates, totalling 350 fits


...

[Parallel(n_jobs=-1)]: Done  76 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 376 tasks      | elapsed:   25.3s
[Parallel(n_jobs=-1)]: Done 876 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 1576 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 2476 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 3208 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 3858 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done 4608 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 5458 tasks      | elapsed:  7.0min
[Parallel(n_jobs=-1)]: Done 6408 tasks      | elapsed:  8.5min
[Parallel(n_jobs=-1)]: Done 7458 tasks      | elapsed: 10.1min
[Parallel(n_jobs=-1)]: Done 8608 tasks      | elapsed: 12.0min
[Parallel(n_jobs=-1)]: Done 9858 tasks      | elapsed: 14.0min
[Parallel(n_jobs=-1)]: Done 11208 tasks      | elapsed: 16.2min
[Parallel(n_jobs=-1)]: Done 12000 out of 12000 | elapsed: 17.7min finished


3_RandomForest | best f1_macro score  47.66% +/- 4.99
best parmams: {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 9, 'max_features': 'sqrt', 'n_estimators': 150}


Result on test dataset with best hyperparameters:
test_acc: 54.60
test_f1_macro: 46.05
test_f1_weighted: 54.25


,0,1,2,3,4,accuracy,macro avg,weighted avg
precision,0.689655,0.400000,0.607595,0.545455,0.076923,0.546012,0.463926,0.572975
recall,0.714286,0.666667,0.657534,0.266667,0.200000,0.546012,0.501031,0.546012
f1-score,0.701754,0.500000,0.631579,0.358209,0.111111,0.546012,0.460531,0.542511
support,28.000000,12.000000,73.000000,45.000000,5.000000,0.546012,163.000000,163.000000


col_0,0,1,2,3,4,All
row_0,,,,,,
0,20,0,6,2,0,28
1,1,8,1,2,0,12
2,4,4,48,6,11,73
3,4,8,20,12,1,45
4,0,0,4,0,1,5
All,29,20,79,22,13,163


----------------------------------------------------------------------------------------------------
Total Time: 0:18:16.001334


In [10]:
print(pd.__version__)
print(sklearn.__version__)
print(np.__version__)

1.3.2
0.23.2
1.19.5
